In [ ]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import boto3
from decimal import Decimal
import time

In [ ]:
with open(Path(r'../access.json'), 'r') as access_file:
    access = json.load(access_file)
    access_file.close()

In [ ]:
test_path = Path(r'../data-test/').glob('*.json')

In [ ]:
data_path = Path(r'../new-data-processed/').glob('*.json')

# use a and acmr as examples

In [ ]:
dynamodb = boto3.resource('dynamodb',region_name='us-east-2',aws_access_key_id=access['KEY_IAM_AWS'],aws_secret_access_key=access['SECRET_IAM_AWS'])

table = dynamodb.Table('Tickers')

In [ ]:
for i in data_path:
    row_dict = {}
    with open(i, 'r') as json_file:
            print(i.stem)
            row_dict['ticker'] = i.stem
            
            data = json.load(json_file)
            
            
            row_dict['price'] = data[i.stem]['price']

            try:
                row_dict['pos_vals'] = data[i.stem]['pos_vals']
            except:
                row_dict['pos_vals'] = None

                print(f"skipping pos_vals for {i}")

            try:
                row_dict['neg_vals'] = data[i.stem]['neg_vals']
            except:
                row_dict['neg_vals'] = None
                
                print(f"skipping neg_vals for {i}")

            try:
                row_dict['dec_vals'] = data[i.stem]['decorrelated_vals']
            except:
                print(f"skipping neg_vals for {i}")
                row_dict['dec_vals'] = None
            table.put_item(Item=json.loads(json.dumps(row_dict), parse_float=Decimal))
            time.sleep(3)
